# Visualizing the training results.
## Model Architecture
- https://www.kaggle.com/code/manojkumars00/intra-image-similarity-with-self-correlation-arch/output

## Baseline Training
- 15 Epochs
- Data Augumentation
- - Horizantal & Vertical Flip
- Metrics
- - IOU : 0.5148
  - Dice Coeff : 0.6797
- https://www.kaggle.com/code/manojkumars00/intra-image-similarity-learning/notebook


## Further Steps Planned
- Synthetic Data Generation
- Trying out Instance Segmentation 

In [ ]:
from intra_image_similarity_with_self_correlation_arch import CmfdModel

model = CmfdModel(encoder_name="nvidia/mit-b1", apply_softmax=False)
model_image_processor = model.image_processor

In [ ]:
import torch
from types import SimpleNamespace

In [ ]:
ckpt_path = "/kaggle/input/intra-image-similarity-learning/checkpoints/best.pt"
checkpoint = torch.load(ckpt_path, map_location="cpu")


In [ ]:
for key, value in checkpoint.items():
    print(key, type(value))

In [ ]:
# Recreate config
cfg = SimpleNamespace(**checkpoint["config"])

state_dict = checkpoint["model"]                      # this is an OrderedDict
missing, unexpected = model.load_state_dict(state_dict, strict=False)
print("Missing keys:", missing)
print("Unexpected keys:", unexpected)

model.eval()

print("Model successfully loaded and ready for inference.")

In [ ]:
import pandas as pd
from PIL import Image
import numpy as np

import matplotlib.pyplot as plt
import torch.nn.functional as F

import os
import random
from skimage import measure

import cv2
from shapely.geometry import Polygon

In [ ]:
def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = np.array(image)
    return image

def load_mask(mask_path):
    mask = np.load(mask_path)
    mask = mask * np.arange(1, mask.shape[0] + 1)[:, None, None]
    mask = mask.sum(axis=0)
    mask = (mask>=1).astype('uint8')
    return mask

In [ ]:
def apply_model_pre_processor(image, mask):
    enc = model_image_processor(
        image,
        segmentation_maps=mask,   
        return_tensors="pt"
    )
    return enc["pixel_values"], enc["labels"].squeeze(0).long()
    

In [ ]:
def plot_sample(image, mask, pred_mask):
    # Read images
    input_img = image
    target_mask = mask
    mask_img = pred_mask

    # assume mask_img[0] is the binary mask
    pred_mask = (pred_mask > 0.5).astype(np.uint8)

    # find contours (boundaries) of forged areas
    contours = measure.find_contours(pred_mask, level=0.5)

    # Plot side-by-side
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 3, 1)
    plt.imshow(input_img)
    plt.title("Model Input Image")
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(target_mask)
    plt.title("Target Mask With Predited Dot Overlay")
    plt.axis('off')

    # draw contours over forged image
    for contour in contours:
        poly = Polygon(contour[:, ::-1]) 
        buffer_distance = 7  # pixels or coordinate units
        buffered_poly = poly.buffer(buffer_distance)
        
        # Convert back to array for plotting
        buffered_contour = np.array(buffered_poly.exterior.coords)
        # plt.plot(contour[:, 1], contour[:, 0], color='red', linewidth=1)
        plt.plot(buffered_contour[:, 0], buffered_contour[:, 1], color='red', linestyle='--')

    plt.subplot(1, 3, 3)
    plt.imshow(pred_mask, cmap='gray')
    plt.title("Predicted Mask")
    plt.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
authentic_images_dir = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images/authentic"
forged_images_dir = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images/forged"
forged_images_mask_dir = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_masks"

In [ ]:
authentic_images_files = sorted(os.listdir(authentic_images_dir))
forged_images_files = sorted(os.listdir(forged_images_dir))
forged_images_mask_files = sorted(os.listdir(forged_images_mask_dir))

# Checking out the how the model learned on training data

In [ ]:
for i in range(25):
    image = load_image(f"{forged_images_dir}/{forged_images_files[i]}")
    mask = load_mask(f"{forged_images_mask_dir}/{forged_images_mask_files[i]}")
    
    preocessed_image, preocessed_mask = apply_model_pre_processor(image, mask)
    
    with torch.no_grad():
        pred_mask = model(preocessed_image)
    
    
    prob_mask = F.softmax(pred_mask, dim=1)
    binary_mask = torch.argmax(prob_mask, dim=1).unsqueeze(0)
    
    resized_mask = F.interpolate(binary_mask.float(), size=(image.shape[0], image.shape[1]), mode='nearest')
    
    resized_pred_mask = resized_mask.squeeze().cpu().numpy()
    
    plot_sample(image, mask, resized_pred_mask)